In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, "/Users/user/git/datasci")
from sci.metrics import rmse_score
from plots import *
import sci.features as scif
import sci.learn as sci
import sci.plots as scip


import pandas as pd
pd.set_option('display.float_format', lambda x: '{:.4f}'.format(x)) #Limiting floats output to
pd.options.display.max_rows = 999

import numpy as np

import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')


/Users/user/miniconda3/envs/test2/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)
Using TensorFlow backend.


## Load Data

In [10]:
# raw data
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

In [11]:
df_train = df_train.set_index("id")
df_test = df_test.set_index("id")

y = df_train["target"]
X = df_train.drop('target', axis=1)

print(y.shape)
print(df_train.shape)
print(X.shape)
print(df_test.shape)

(250,)
(250, 301)
(250, 300)
(19750, 300)


In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_proc = scaler.fit_transform(X)
df_test_proc = scaler.transform(df_test)

X_proc = pd.DataFrame(X_proc, index=X.index, columns=X.columns)
df_test_proc = pd.DataFrame(df_test_proc, index=df_test.index, columns=df_test.columns)

## Save

In [13]:
path = 'processed_min'
!mkdir -p $path
X_proc.to_msgpack(f'{path}/X.msgpack')
y.to_msgpack(f'{path}/y.msgpack')
df_test_proc.to_msgpack(f'{path}/X_score.msgpack')

# from sklearn.externals import joblib
# joblib.dump(y_tr, f'{path}/y_tr.joblib')


## Quick Test

In [14]:
import sklearn.utils
from sklearn.linear_model import *
import sci
from sklearn.metrics import roc_auc_score, make_scorer

In [15]:
import xgboost
from sklearn.svm import SVC, LinearSVC, NuSVC

# model = ElasticNetCV(l1_ratio=0.5)
# model = xgboost.XGBClassifier()
model = SVC(kernel="linear", C=0.025, probability=True)

sci.metric_global = roc_auc_score
sci.scoring_global = make_scorer(sci.metric_global, needs_proba=True)

data = sci.load_data(path)
cv_scores = sci.score_cv(data, model, scoring=sci.scoring_global, verbose=2, cv=10)

(250, 300) (250,)
CV for SVC ..


[Parallel(n_jobs=11)]: Using backend LokyBackend with 11 concurrent workers.
[Parallel(n_jobs=11)]: Done   7 out of  10 | elapsed:    2.0s remaining:    0.9s
[Parallel(n_jobs=11)]: Done  10 out of  10 | elapsed:    2.0s finished


,mean,std
SVC,0.7444,0.0311


Can't plot with only a single or no samples
